# Machine Learning Exercises in Python
# Part 4: Multivariate Logistic Regression

http://www.johnwittenauer.net/machine-learning-exercises-in-python-part-4/

In [31]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

%matplotlib inline
%cd ~

ex_dir = lambda n: './courses/Machine Learning/machine-learning-ex{}/ex{}'.format(n, n)

/Users/joanne


## Visualising the data

In [33]:
path = os.path.join(ex_dir(3), 'ex3data1.mat')
data = loadmat(path)
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

There are 5000 training examples in `ex3data1.mat`, where each training example is a 20 pixel by 20 pixel grayscale image of the digit. Each pixel is represented by a floating point number indicating the grayscale intensity at that location. The 20 by 20 grid of pixels is “unrolled” into a 400-dimensional vector. Each of these training examples becomes a single row in our data matrix `X`.

The second part of the training set is a 5000-dimensional vector `y` that contains labels for the training set, that is, digits. The digit zero is mapped to the value ten.

In [283]:
X, y = data['X'], data['y']
X.shape, y.shape
# TODO: Try to actually visualise the data

((5000, 400), (5000, 1))

## Sigmoid, cost and gradient descent functions

In [284]:
X = np.matrix(X)
y = np.matrix(y)

In [285]:
if X.shape[1] <= 400:
    before = X.shape
    X = np.insert(X, 0, values=np.ones(X.shape[0]), axis=1)
    after = X.shape
    print('X padded with 0s {} -> {}'.format(before, after))
else:
    print('X already padded')

X padded with 0s (5000, 400) -> (5000, 401)


In [334]:
def sigmoid(z):
    return 1. / (1. + np.exp(-z))

# Let's take matrices in
def cost(theta, X, y, learning_rate):
    m = X.shape[0]
    h_x = sigmoid(X * theta.T)
    first = np.multiply(-y, np.log(h_x))
    second = np.multiply((1 - y), np.log(1 - h_x))
    reg = (learning_rate / (2*m)) * np.sum(theta * theta.T)
    return (1 / m) * np.sum(first - second) + reg

In [335]:
theta = np.matrix(np.zeros(X.shape[1]))
cost(theta, X, y, 1)

-17.051420641774662

In [336]:
def gradient_with_loop(theta, X, y, learning_rate):
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)

    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learning_rate / len(X)) * theta[:,i])
    return grad

In [337]:
def gradient(theta, X, y, learning_rate):
    error = sigmoid(X * theta.T) - y
    m = len(X)
    grad = (error.T * X) / m
    reg = (learning_rate / m) * ([0] + np.hstack([[[0]], theta[:,1:]]))
    grad += reg
    return np.asarray(grad).ravel()

In [353]:
theta_test = np.matrix(np.random.randint(0, 10 + 1, X.shape[1]))
print('With for loop:', gradient_with_loop(theta_test, X, y, 1).sum())
print('Vectorised:', gradient(theta_test, X, y, 1).sum())

With for loop: -255.287455748
Vectorised: -255.287455748


## One vs. all classifier

In [356]:
from scipy.optimize import minimize

# Write some wrappers to match formats used by the scipy.optimize function
cost_wrapper = lambda theta, X, y, lr: cost(np.matrix(theta), X, y, lr)
gradient_wrapper = lambda theta, X, y, lr: gradient(np.matrix(theta), X, y, lr)

def one_vs_all(X, y, num_labels, learning_rate):
    m, n = X.shape
    
    # k * (n+1) for parameters of each classifier
    all_theta = np.zeros((num_labels, n))
    
    for i in range(1, num_labels + 1):
        theta_tmp = np.zeros(n)
        y_i = np.array([1 if label == i else 0 for label in y]).reshape((m, 1))
        
        # minimise the objective function
        fmin = minimize(
            fun=cost_wrapper, 
            x0=theta_tmp, 
            args=(X, y_i, learning_rate), 
            method='TNC', 
            jac=gradient_wrapper)
        
        all_theta[i-1,:] = fmin.x
        
    return all_theta
    

In [357]:
%%time
all_theta = one_vs_all(X, y, 10, 1)

CPU times: user 14.4 s, sys: 326 ms, total: 14.7 s
Wall time: 9.4 s


## Predicting labels

In [359]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [361]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a,b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct))) / float(len(correct))
print('Training set accuracy = {0}%'.format(accuracy * 100))

Training set accuracy = 94.42%
